In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta

import hopsworks
# from hsfs.feature_store import FeatureStore
import pandas as pd
import numpy as np

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.component.feature_group_config as config
from src.component.feature_store_api import get_feature_store, get_or_create_feature_view
from src.component.feature_group_config import FEATURE_VIEW_METADATA

In [3]:
current_date = pd.to_datetime(datetime.utcnow(),utc=True).floor('h')
current_date

Timestamp('2025-04-05 13:00:00+0000', tz='UTC')

In [4]:
#load recent batch of data from feature store
from src.component.inference import load_batch_of_features_from_store
features = load_batch_of_features_from_store(current_date)

HOPSWORKS_PROJECT_NAME:  energy_demand_prediction
2025-04-05 18:56:26,112 INFO: Initializing external client
2025-04-05 18:56:26,113 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-05 18:56:27,849 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-05 18:56:29,730 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213640
2025-04-05 18:56:34,302 INFO: Feature view already exists, skipping creation.
HOPSWORKS_PROJECT_NAME:  energy_demand_prediction
2025-04-05 18:56:34,305 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-05 18:56:34,316 INFO: Initializing external client
2025-04-05 18:56:34,319 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-04-05 18:56:36,967 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213640
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.68s) 
Dates before filtering:
2025-03-07 13:00:00+00:00 2025-04-05 12:00:00+00:00
                          date  sub_region_code demand  temperature_2m  \
7513 2025-03-29 14:00:00+00:00                3    690       21.758501   
7515 2025-03-11 00:00:00+00:00                7    365        3.708500   
7523 2025-03-07 18:00:00+00:00                3    642        3.858500   
7527 2025-03-09 20:00:00+00:00                7    295        9.708500   
7528 2025-03-19 13:00:00+00:00                2   1826       15.758500   
...                        ...              ...    ...             ...   
7651 2025-03-17 11:00:00+00:00                1   1097        5.408500   
7652 2025-03-12 14:00:00+00:00               10   1860       13.408501   
7653 2025-03-24 19:00:00+00:00     

In [5]:
features

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_7_hour,demand_previous_6_hour,demand_previous_5_hour,demand_previous_4_hour,demand_previous_3_hour,demand_previous_2_hour,demand_previous_1_hour,temperature_2m,date,sub_region_code
0,1801.0,1759.0,1689.0,1633.0,1593.0,1539.0,1572.0,1559.0,1604.0,1689.0,...,1564.0,1535.0,1522.0,1526.0,1549.0,1587.0,1617.0,20.8085,2025-04-05 13:00:00+00:00,0
1,1097.0,1008.0,948.0,854.0,777.0,757.0,796.0,957.0,1051.0,1099.0,...,905.0,891.0,879.0,884.0,903.0,933.0,942.0,20.8085,2025-04-05 13:00:00+00:00,1
2,1776.0,1606.0,1488.0,1400.0,1346.0,1311.0,1277.0,1256.0,1320.0,1504.0,...,1524.0,1515.0,1507.0,1508.0,1534.0,1586.0,1624.0,20.8085,2025-04-05 13:00:00+00:00,2
3,700.0,685.0,666.0,641.0,642.0,639.0,623.0,626.0,636.0,655.0,...,664.0,658.0,646.0,661.0,664.0,673.0,670.0,20.8085,2025-04-05 13:00:00+00:00,3
4,823.0,683.0,604.0,570.0,565.0,535.0,507.0,501.0,521.0,638.0,...,723.0,720.0,709.0,712.0,731.0,768.0,781.0,20.8085,2025-04-05 13:00:00+00:00,4
5,1226.0,1069.0,950.0,898.0,878.0,869.0,868.0,920.0,1061.0,1303.0,...,1067.0,1047.0,1047.0,1057.0,1082.0,1119.0,1150.0,20.8085,2025-04-05 13:00:00+00:00,5
6,887.0,779.0,735.0,698.0,666.0,653.0,670.0,763.0,909.0,1042.0,...,904.0,888.0,880.0,880.0,900.0,939.0,964.0,20.8085,2025-04-05 13:00:00+00:00,6
7,305.0,298.0,288.0,273.0,264.0,258.0,249.0,252.0,265.0,283.0,...,283.0,276.0,274.0,274.0,279.0,299.0,309.0,20.8085,2025-04-05 13:00:00+00:00,7
8,609.0,598.0,580.0,571.0,559.0,546.0,542.0,549.0,568.0,591.0,...,508.0,492.0,484.0,481.0,493.0,511.0,532.0,20.8085,2025-04-05 13:00:00+00:00,8
9,5228.0,5426.0,5519.0,5510.0,5516.0,5482.0,5452.0,5437.0,5500.0,5572.0,...,4418.0,4279.0,4205.0,4186.0,4251.0,4381.0,4550.0,20.8085,2025-04-05 13:00:00+00:00,9


In [6]:
from src.component.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

2025-04-05 18:56:49,430 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-05 18:56:49,442 INFO: Initializing external client
2025-04-05 18:56:49,444 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-05 18:56:51,195 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-05 18:56:52,326 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213640
[LightGBM] [Warning] feature_fraction is set=0.9501545297247309, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9501545297247309
[LightGBM] [Warning] bagging_fraction is set=0.4518490719321875, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4518490719321875


In [7]:
predictions['date'] = current_date
#predictions['seconds'] = predictions['date'].astype('int64') // 10**6
predictions

,sub_region_code,predicted_demand,date
0,0,1600.0,2025-04-05 13:00:00+00:00
1,1,963.0,2025-04-05 13:00:00+00:00
2,2,1637.0,2025-04-05 13:00:00+00:00
3,3,670.0,2025-04-05 13:00:00+00:00
4,4,777.0,2025-04-05 13:00:00+00:00
5,5,1198.0,2025-04-05 13:00:00+00:00
6,6,979.0,2025-04-05 13:00:00+00:00
7,7,307.0,2025-04-05 13:00:00+00:00
8,8,533.0,2025-04-05 13:00:00+00:00
9,9,4576.0,2025-04-05 13:00:00+00:00


In [8]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=6,
    features=features,
    targets=predictions['predicted_demand'],
    predictions=pd.Series(predictions['predicted_demand'])
)

In [9]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=8,
    features=features,
    targets=predictions['predicted_demand'],
    predictions=pd.Series(predictions['predicted_demand'])
)

In [10]:
#save predictions to the feature store
from src.component.feature_store_api import get_feature_store
import src.component.feature_group_config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['sub_region_code', 'date'],
    event_time='date',
)



HOPSWORKS_PROJECT_NAME:  energy_demand_prediction
2025-04-05 18:40:09,321 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-05 18:40:09,330 INFO: Initializing external client
2025-04-05 18:40:09,331 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-05 18:40:11,056 WARNING: c:\Users\HP\anaconda3\envs\credit_card_approval\Lib\site-packages\hopsworks_common\connection.py:310: UserWarning:

The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'




2025-04-05 18:40:14,234 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213640


In [7]:
predictions

,sub_region_code,predicted_demand
0,0,1600.0
1,1,963.0
2,2,1637.0
3,3,670.0
4,4,777.0
5,5,1198.0
6,6,979.0
7,7,307.0
8,8,533.0
9,9,4576.0


In [11]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213640/fs/1200277/fg/1434989


Uploading Dataframe: 100.00% |██████████| Rows 11/11 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213640/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)